In [1]:
from __future__ import division
import os, time

In [2]:
#pip install Pillow

In [3]:
import PIL

In [4]:
from PIL import Image
#Image.fromarray(data)

In [5]:
import scipy as scipy

In [6]:
scipy.__version__

'1.7.3'

In [7]:
#from scipy.stats import betabinom

In [8]:
import tensorflow.compat.v1 as tf

In [9]:
pip install --upgrade tf_slim

Note: you may need to restart the kernel to use updated packages.


In [10]:
import tf_slim as slim

In [11]:
#import tensorflow.contrib.slim as slim
import numpy as np
#import rawpy


In [12]:
pip install rawpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import rawpy as rawpy

In [14]:
import glob

In [15]:
#from google.colab import drive
#drive.mount('/content/drive',force_remount = True)

In [16]:
input_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/dataset/Fuji/Fuji/short/'
gt_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/dataset/Fuji/Fuji/long/'
checkpoint_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/'
result_dir = '/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/result4_Fuji/'

In [17]:
import sys

In [18]:
# get train IDs
train_fns = glob.glob(gt_dir + '0*.RAF')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

ps = 512  # patch size for training
save_freq = 1

In [19]:
len(train_ids)

135

In [20]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)


def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.random.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output

In [21]:
def network(input):  # Unet
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 27, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 3)
    return out

In [22]:
def pack_raw(raw):
    # pack X-Trans image to 9 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 1024, 0) / (16383 - 1024)  # subtract the black level

    img_shape = im.shape
    H = (img_shape[0] // 6) * 6
    W = (img_shape[1] // 6) * 6

    out = np.zeros((H // 3, W // 3, 9))

    # 0 R
    out[0::2, 0::2, 0] = im[0:H:6, 0:W:6]
    out[0::2, 1::2, 0] = im[0:H:6, 4:W:6]
    out[1::2, 0::2, 0] = im[3:H:6, 1:W:6]
    out[1::2, 1::2, 0] = im[3:H:6, 3:W:6]

    # 1 G
    out[0::2, 0::2, 1] = im[0:H:6, 2:W:6]
    out[0::2, 1::2, 1] = im[0:H:6, 5:W:6]
    out[1::2, 0::2, 1] = im[3:H:6, 2:W:6]
    out[1::2, 1::2, 1] = im[3:H:6, 5:W:6]

    # 1 B
    out[0::2, 0::2, 2] = im[0:H:6, 1:W:6]
    out[0::2, 1::2, 2] = im[0:H:6, 3:W:6]
    out[1::2, 0::2, 2] = im[3:H:6, 0:W:6]
    out[1::2, 1::2, 2] = im[3:H:6, 4:W:6]

    # 4 R
    out[0::2, 0::2, 3] = im[1:H:6, 2:W:6]
    out[0::2, 1::2, 3] = im[2:H:6, 5:W:6]
    out[1::2, 0::2, 3] = im[5:H:6, 2:W:6]
    out[1::2, 1::2, 3] = im[4:H:6, 5:W:6]

    # 5 B
    out[0::2, 0::2, 4] = im[2:H:6, 2:W:6]
    out[0::2, 1::2, 4] = im[1:H:6, 5:W:6]
    out[1::2, 0::2, 4] = im[4:H:6, 2:W:6]
    out[1::2, 1::2, 4] = im[5:H:6, 5:W:6]

    out[:, :, 5] = im[1:H:3, 0:W:3]
    out[:, :, 6] = im[1:H:3, 1:W:3]
    out[:, :, 7] = im[2:H:3, 0:W:3]
    out[:, :, 8] = im[2:H:3, 1:W:3]
    return out

In [23]:
init_op = tf.initialize_all_variables()
sess = tf.Session()

tf.disable_eager_execution()
in_image = tf.placeholder(tf.float32, [None, None, None, 9]) #Just a placeholder for assigning the values in further code
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])

tf.disable_v2_behavior()
# Change it to the sample expression as follows. 
#init = tf.random.truncated_normal()
#sess.run(init_op)
out_image = network(in_image)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
non-resource variables are not supported in the long term


2022-10-09 23:57:05.583859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Applications/Anaconda/anaconda3/envs/tf/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [24]:
G_loss = tf.reduce_mean(tf.abs(out_image - gt_image)) #Computes the mean of elements across dimensions of a tensor

t_vars = tf.trainable_variables() #Returns all variables created with trainable=True
lr = tf.placeholder(tf.float32) #Place for storing learning rate
G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir) #Accessing checkpoint state from the ckpt directory
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path) #Restoring the parameters saved in the checkpoint path

2022-10-09 23:57:06.240437: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [25]:
# Raw data takes long time to load. Keep them in memory after loaded.
gt_images = [None] * 6000
in_images = {}
in_images['300'] = [None] * len(train_ids)
in_images['250'] = [None] * len(train_ids)
in_images['100'] = [None] * len(train_ids)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
import scipy.misc

In [28]:
np.random.seed(10)
learning_rate = 1e-4
for epoch in range(lastepoch, 5):
    if os.path.isdir(result_dir + '%04d' % epoch):
        continue
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5

    for ind in np.random.permutation(len(train_ids)):
        #try: 
            # get the path from image id
            train_id = train_ids[ind]
            in_files = glob.glob(input_dir + '%05d_00*.RAF' % train_id)
            in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
            in_fn = os.path.basename(in_path)

            gt_files = glob.glob(gt_dir + '%05d_00*.RAF' % train_id)
            gt_path = gt_files[0]
            gt_fn = os.path.basename(gt_path)
            in_exposure = float(in_fn[9:-5])
            gt_exposure = float(gt_fn[9:-5])
            ratio = min(gt_exposure / in_exposure, 300)

            st = time.time()
            cnt += 1

            if in_images[str(ratio)[0:3]][ind] is None:

                raw = rawpy.imread(in_path)
                in_images[str(ratio)[0:3]][ind] = np.expand_dims(pack_raw(raw), axis=0) * ratio

                gt_raw = rawpy.imread(gt_path)
                im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)

            # crop
            H = in_images[str(ratio)[0:3]][ind].shape[1]
            W = in_images[str(ratio)[0:3]][ind].shape[2]
 
            xx = np.random.randint(0, W - ps)
            yy = np.random.randint(0, H - ps)
            input_patch = in_images[str(ratio)[0:3]][ind][:, yy:yy + ps, xx:xx + ps, :]
            gt_patch = gt_images[ind][:, yy * 3:yy * 3 + ps * 3, xx * 3:xx * 3 + ps * 3, :]

            if np.random.randint(2, size=1)[0] == 1:  # random flip
                input_patch = np.flip(input_patch, axis=1)
                gt_patch = np.flip(gt_patch, axis=1)
            if np.random.randint(2, size=1)[0] == 1:
                input_patch = np.flip(input_patch, axis=2)
                gt_patch = np.flip(gt_patch, axis=2)
            if np.random.randint(2, size=1)[0] == 1:  # random transpose
                input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

            input_patch = np.minimum(input_patch, 1.0)

            _, G_current, output = sess.run([G_opt, G_loss, out_image],
                                          feed_dict={in_image: input_patch, gt_image: gt_patch, lr: learning_rate})
            output = np.minimum(np.maximum(output, 0), 1)
            g_loss[ind] = G_current

            print('%d Count: %d Train ID: %d Loss=%.3f, Time=%.3f' % (epoch, cnt,train_id, np.mean(g_loss[np.where(g_loss)]), time.time() - st))

            if epoch % save_freq == 0:
                if not os.path.isdir(result_dir + '%04d' % epoch):
                    os.makedirs(result_dir + '%04d' % epoch)

                temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :, :]), axis=1)
                Image.fromarray((temp*255).astype(np.uint8),mode='RGB').save(result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))
                #scipy.misc.toimage(temp * 255, high=255, low=0, cmin=0, cmax=255).save(result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))
            if cnt>=15:
                break
        #except:
        #    print(f"Oops! {sys.exc_info()[0]} occurred for Train ID {train_id}, moving to next training ID....")
    if not os.path.isdir(result_dir + 'model.ckpt'):
        os.makedirs(result_dir + 'model.ckpt')
    saver.save(sess, checkpoint_dir + 'model.ckpt')

0 Count: 1 Train ID: 186 Loss=0.134, Time=17.658
0 Count: 2 Train ID: 130 Loss=0.081, Time=17.764
0 Count: 3 Train ID: 192 Loss=0.133, Time=17.831
0 Count: 4 Train ID: 185 Loss=0.120, Time=18.356
0 Count: 5 Train ID: 182 Loss=0.137, Time=17.444
0 Count: 6 Train ID: 107 Loss=0.124, Time=17.368
0 Count: 7 Train ID: 164 Loss=0.132, Time=17.476
0 Count: 8 Train ID: 170 Loss=0.145, Time=18.425
0 Count: 9 Train ID: 75 Loss=0.152, Time=18.169
0 Count: 10 Train ID: 2 Loss=0.155, Time=18.698
0 Count: 11 Train ID: 32 Loss=0.154, Time=17.673
0 Count: 12 Train ID: 111 Loss=0.163, Time=18.019
0 Count: 13 Train ID: 152 Loss=0.157, Time=16.465
0 Count: 14 Train ID: 131 Loss=0.167, Time=16.826
0 Count: 15 Train ID: 180 Loss=0.181, Time=17.605
INFO:tensorflow:/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/model.ckpt.meta
INFO:tensorflow:400
INFO:tensorflow:/Users/rahulvaishnav/Downloads/IITB_IEOR/Sem_3/IE643/Course_Project/Fuji_Training/ckpt4_Fuji/model.ckp